In [1]:
import json
from tqdm import tqdm
import re
import torch

# 定义 color_mapping
color_mapping = {
    'red': [0.1, 0.15],
    'green': [0.15, 0.3],
    'blue': [0.3, 0.45],
    'yellow': [0.45, 0.6],
    'orange': [0.6, 0.75],
    'purple': [0.75, 0.9]
}

# 提取所有颜色
colors = color_mapping.keys()

# 检查颜色是否存在于字符串中
def find_colors_in_string(input_string):
    found_colors = [color for color in colors if color in input_string]
    return found_colors


def map_to_color(pixel):
    if pixel<0.1:
        return 'black'
    elif 0.1<=pixel<0.15:
        return 'red'
    elif 0.15<=pixel<0.3:
        return 'green'
    elif 0.3<=pixel<0.45:
        return 'blue'
    elif 0.45<=pixel<0.6:
        return 'yellow'
    elif 0.6<=pixel<0.75:
        return 'orange'
    elif 0.75<=pixel<=0.9:
        return 'purple'
    else:
        return 'other'

def compare_img(gen_img,gt_img):
    correct_pixel=0
    incorrect_pixel=0
    for i in range(len(gen_img)):
        for j in range(len(gen_img[i])):
            if map_to_color(gen_img[i][j])!=map_to_color(gt_img[i][j]):
                incorrect_pixel+=1
            else:
                correct_pixel+=1
    return correct_pixel,incorrect_pixel

acc=[]
answer_list=[i*78 for i in range(1,6)]
for answer in answer_list:
    #data_path=f'/datadrive_a/jihai/LLaVA/answer/answer-llava-v1.5-7b-mix-u-odd-{i}.jsonl'
    data_path=f'./answer-llava-v1.5-7b-segment-digit-lora-{answer}.jsonl'
    correct_piexl=0
    incorrect_pixel=0
    correct_time=0
    incorrect_time=0
    correct_color=0
    incorrect_color=0
    count=0
    bound=5
    with open (data_path, "r") as f:
        for line in tqdm(f):
            # count+=1
            # if count<bound:
            #     continue
            # if count>bound:
            #     break
            json_obj = json.loads(line.strip())
            #print(json_obj)
            # generation:
            if '<image>' in json_obj['groun_truth']:
                gt_img=json_obj['groun_truth_img_tensor']
                gen_img=json_obj['output_img_tensor']
                if gen_img !=None:
                    correct,incorrect=compare_img(gen_img,gt_img)
                    correct_piexl+=correct
                    incorrect_pixel+=incorrect
                else:
                    incorrect_pixel+=6*7
            else:
                gt_time=re.findall(r'\d+',json_obj['groun_truth'])
                gen_time=re.findall(r'\d+',json_obj['answer'])
                if len(gen_time)!=0:
                    for i in range(len(gt_time)):
                        if i<len(gen_time) and gt_time[i]==gen_time[i]:
                            correct_time+=1
                        else:
                            incorrect_time+=1
                else:
                    incorrect_time+=3
                gt_color=find_colors_in_string(json_obj['groun_truth'])
                gen_color=find_colors_in_string(json_obj['answer'])
                if len(gen_color)!=0 and gt_color[0]==gen_color[0]:
                    correct_color+=1
                else:
                    incorrect_color+=1
    acc.append([correct_piexl/(correct_piexl+incorrect_pixel),correct_time/(correct_time+incorrect_time),correct_color/(correct_color+incorrect_color)])
    print(f"generation_accuracy:{correct_piexl/(correct_piexl+incorrect_pixel)}")
    print(f"time_accuracy:{correct_time/(correct_time+incorrect_time)}")
    print(f"color_accuracy:{correct_color/(correct_color+incorrect_color)}")
acc=torch.Tensor(acc)
acc=acc.permute(1,0)
print(acc[0])
print(acc[1])
print(acc[2])


200it [00:00, 10804.77it/s]


generation_accuracy:0.3333333333333333
time_accuracy:0.051587301587301584
color_accuracy:0.9523809523809523


200it [00:00, 5660.56it/s]


generation_accuracy:0.5242200328407225
time_accuracy:0.027777777777777776
color_accuracy:0.7261904761904762


200it [00:00, 10589.67it/s]


generation_accuracy:0.7830459770114943
time_accuracy:0.8333333333333334
color_accuracy:0.8690476190476191


200it [00:00, 5518.28it/s]


generation_accuracy:0.9113300492610837
time_accuracy:1.0
color_accuracy:0.9761904761904762


138it [00:00, 5153.49it/s]

generation_accuracy:0.9206349206349206
time_accuracy:1.0
color_accuracy:0.9473684210526315
tensor([0.3333, 0.5242, 0.7830, 0.9113, 0.9206])
tensor([0.0516, 0.0278, 0.8333, 1.0000, 1.0000])
tensor([0.9524, 0.7262, 0.8690, 0.9762, 0.9474])


In [12]:
print(':'.join(gt_time))

18:14:20


In [8]:
from sklearn import kernel_approximation
import torch
import torch.nn.functional as F
padded_input=torch.Tensor([[0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 1., 1., 0.]]).unsqueeze(0).unsqueeze(0)
padded_input=torch.Tensor([[0., 1., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0.]]).unsqueeze(0).unsqueeze(0)
kernel=torch.Tensor([[1, 0, 3],
    [0, 0, 2],
    [4, 1, 0]]).unsqueeze(0).unsqueeze(0) 
output = F.conv2d(padded_input, kernel, stride=1)
print(output)

tensor([[[[4., 9., 5., 4.],
          [2., 1., 4., 2.],
          [6., 6., 6., 7.],
          [5., 2., 1., 2.]]]])
